In [1]:
import numpy as np
import pandas as pd

from basic.mnist import Mnist

In [2]:
# load data from Mnist Dataset
dataloader = Mnist()
train_data, train_label = dataloader.get_data(train=True)
test_data, test_label = dataloader.get_data(False)

In [3]:
train_data

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
np.array(train_data).shape

(64, 784)

### Loss function of Logistic Regression:
$$
\scalebox{1.5}{
\begin{align*}
J(\theta) = -\frac{1}{m} \sum_{i=1}^{m} \left[ y^{(i)} \log\left(h_\theta(x^{(i)})\right) + (1 - y^{(i)}) \log\left(1 - h_\theta(x^{(i)})\right) \right]
\end{align*}
}
$$

### Gradient Calculation:
$$\scalebox{1.5} {
\begin{align*}
g_i = \frac{\partial J(\theta)} {\partial w_i}=(p(x_i) - y_i)x_i
\end{align*}
}
$$

### how to calculate p(x_i)
$$\scalebox{1.5} {
\begin{align*}
p(x_i) = \frac{1}{1 + e^{-(w^Tx + b)}}
\end{align*}
}
$$

In [5]:
def logistic_regression(training_data: pd.DataFrame, training_labels:pd.DataFrame, epochs=10, step = 0.001):
    """
    :param training_data: 训练数据
    :param training_labels: 训练数据标签
    :param epochs: 迭代次数
    :param step: 步长
    :return:
    """
    train_data_arr = training_data.values
    train_label_arr = np.array(training_labels)

    train_data_arr = np.c_[train_data_arr, np.ones((train_data_arr.shape[0], 1))]  
    
    # 初始化w,
    w = np.zeros(train_data_arr.shape[1])

    # 迭代轮数
    for epoch in range(epochs):
        # 遍历数据进行调参
        if epoch % 10 == 0:
            print(epoch)
        for idx in range(train_data_arr.shape[0]):
            x_i = train_data_arr[idx]
            y_i = train_label_arr[idx]
            wx = np.dot(w, x_i)
            # 梯度下降
            px_i = 1 / (1 + np.exp(-wx))
            gradient = (px_i - y_i) * x_i
            w += step * gradient

    return w

In [6]:
def pred(w, x):
    wx = np.dot(w, x)
    p = np.exp(wx) / (1 + np.exp(wx))
    if p > 0.5:
        return 1
    else:
        return 0

In [11]:
def test(w, test_data:pd.DataFrame, test_label: pd.DataFrame):
    
    err_cnt = 0

    # 将偏置项添加到测试数据中
    test_data_with_bias = np.c_[test_data, np.ones((test_data.shape[0], 1))]
    
    test_label = np.array(test_label)
    
    for idx in range(len(test_data_with_bias)):
        # 使用 pred 函数获取预测结果
        prediction = pred(w, test_data_with_bias[idx])
        # 比较预测结果和实际标签，更新错误计数
        if test_label[idx] != prediction:
            err_cnt += 1

    # 计算准确率
    accuracy = 1 - err_cnt / len(test_data)
    return round(accuracy, 2)

In [8]:
w = logistic_regression(train_data, train_label)

0


In [9]:
w.shape

(785,)

In [12]:
test(w, test_data, test_label)

0.03